In [1]:
# Cell 1: imports and setup
import os
from typing import Optional
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# display and plotting setup
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 160)
plt.rcParams["figure.figsize"] = (10, 5)

# default paths (adjust if needed)
COMPANIES_PATH = r"../data/raw/acoes-listadas-b3.csv"
OUTPUT_DIR = r"../data/processed"

In [2]:
# Cell 2: load B3-listed companies (CSV validation included)
def load_listed_companies(path: str = COMPANIES_PATH) -> Optional[pd.DataFrame]:
    try:
        df = pd.read_csv(path)
    except FileNotFoundError:
        print(f"❌ File not found: {path}")
        return None
    except Exception as e:
        print(f"❌ Error while reading CSV: {e}")
        return None

    # expected column: 'Ticker'
    if "Ticker" not in df.columns:
        possible = [c for c in df.columns if "tick" in c.lower() or "code" in c.lower()]
        if possible:
            print("⚠️ Missing 'Ticker' column — maybe you meant:", possible)
        else:
            print("⚠️ 'Ticker' column not found in CSV.")
        return None

    print(f"✅ Loaded {len(df)} tickers.")
    return df

In [3]:
# Cell 3: load and preview the first few rows
df_listed = load_listed_companies()
if df_listed is not None:
    display(df_listed.head(10))
    print("\nColumns:", df_listed.columns.tolist())

✅ Loaded 406 tickers.


,Ticker,Nome,Negócios,Última (R$),Variação
0,PDGR3,PDG Realty,34.615.600,"0,07","-46,15%"
1,MGLU3,Magazine Luiza,32.721.500,"9,15","-0,11%"
2,AMBP3,Ambipar,27.231.800,"1,40","-49,09%"
3,ASAI3,Assaí,24.871.500,"9,06","-2,16%"
4,ITSA4,Itaúsa,22.795.700,"11,22","+0,54%"
5,BBDC4,Banco Bradesco,22.193.300,"17,10","+0,06%"
6,ABEV3,Ambev,21.785.000,"11,87","-0,50%"
7,COGN3,Cogna,21.087.500,"3,21","+1,58%"
8,PETR4,Petrobras,19.756.800,"31,00","-0,26%"
9,VALE3,Vale,19.240.200,"58,59","-0,19%"



Columns: ['Ticker', 'Nome', 'Negócios', 'Última (R$)', 'Variação']


In [4]:
# Cell 4: select a ticker (user input or auto-pick)
def choose_ticker(df: pd.DataFrame, interactive: bool = True) -> Optional[str]:
    tickers = df["Ticker"].astype(str).str.upper().unique()

    # auto mode: pick the first one
    if not interactive:
        print(f"Non-interactive mode: using {tickers[0]}")
        return tickers[0] + ".SA"

    # interactive mode
    print("Example tickers (first 20):")
    print(", ".join(tickers[:20]))
    ticker = input("Enter ticker (no .SA) or index (e.g. 0): ").strip().upper()

    if ticker.isdigit():
        idx = int(ticker)
        if 0 <= idx < len(tickers):
            return tickers[idx] + ".SA"
        else:
            print("Index out of range.")
            return None

    if ticker in tickers:
        return ticker + ".SA"

    print("Ticker not found.")
    return None

# usage:
ticker = choose_ticker(df_listed, interactive=True)


Example tickers (first 20):
PDGR3, MGLU3, AMBP3, ASAI3, ITSA4, BBDC4, ABEV3, COGN3, PETR4, VALE3, ITUB4, RAIZ4, BBAS3, CSAN3, MOTV3, B3SA3, MBRF3, RENT3, GGBR4, PCAR3


In [5]:
# Cell 5: baixar histórico completo para 1 ticker
def download_full_history(ticker: str, timeout: int = 20) -> Optional[pd.DataFrame]:
    try:
        print(f"⬇️ Baixando {ticker} ...")
        data = yf.download(
            tickers=ticker,
            period="max",
            interval="1d",
            auto_adjust=True,
            progress=True,
            threads=True,
            group_by="column",
            timeout=timeout
        )
        if data.empty:
            print("⚠️ Nenhum dado retornado.")
            return None
        # se houver multiindex de colunas (quando várias ações), normaliza:
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.droplevel(0)
        data.index = pd.to_datetime(data.index)
        print(f"✅ {len(data)} registros de {data.index.min().date()} a {data.index.max().date()}")
        return data
    except Exception as e:
        print(f"❌ Erro no download: {type(e).__name__} - {e}")
        return None

# exemplo de uso:
df_hist = download_full_history(ticker)
df_hist.columns = ["Open", "High", "Low", "Close", "Volume"]


⬇️ Baixando PETR4.SA ...


[*********************100%***********************]  1 of 1 completed

✅ 6476 registros de 2000-01-03 a 2025-10-15


In [6]:
df_hist.reset_index(inplace=True)
df_hist.drop(columns=['Date'], inplace=True)


In [7]:
import pandas as pd
import numpy as np

def mark_and_remove_outliers(df, multiplier=1.5, verbose=True):
    """
    Para cada coluna numérica:
      - Cria coluna <nome_coluna>_outlier: 1 se for outlier, 0 caso contrário
      - Remove todas as linhas que forem outliers em qualquer coluna
      - Remove as colunas de outlier antes de retornar
    """
    df_clean = df.copy()
    
    # Flatten MultiIndex se houver
    if isinstance(df_clean.columns, pd.MultiIndex):
        df_clean.columns = ['_'.join(map(str, col)).strip() for col in df_clean.columns.values]

    num_cols = df_clean.select_dtypes(include=[np.number]).columns
    if len(num_cols) == 0:
        if verbose:
            print("⚠️ Nenhuma coluna numérica encontrada")
        return df_clean

    outlier_cols = []
    for col in num_cols:
        col_data = df_clean[col]
        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - multiplier * IQR
        upper = Q3 + multiplier * IQR

        # cria coluna de outlier
        outlier_col = f"{col}_outlier"
        df_clean[outlier_col] = ((col_data < lower) | (col_data > upper)).astype(int)
        outlier_cols.append(outlier_col)

    # remove todas as linhas que tiverem 1 em qualquer coluna de outlier
    before = len(df_clean)
    mask = df_clean[outlier_cols].sum(axis=1) == 0
    df_clean = df_clean[mask]
    after = len(df_clean)

    # drop colunas de outlier
    df_clean.drop(columns=outlier_cols, inplace=True)

    if verbose:
        removed = before - after
        print(f"Outlier removal completed:")
        print(f"  Columns checked: {len(num_cols)}")
        print(f"  Rows before: {before}")
        print(f"  Rows after:  {after}")
        print(f"  Removed:     {removed} ({removed/before*100:.2f}%)")

    return df_clean

df_hist = mark_and_remove_outliers(df_hist)


Outlier removal completed:
  Columns checked: 5
  Rows before: 6476
  Rows after:  5404
  Removed:     1072 (16.55%)


In [8]:
# Cell 7: export to CSV with fixed name "df.csv"
def export_data(data: pd.DataFrame, output_dir: str = OUTPUT_DIR):
    """
    Export the DataFrame to a CSV file with a fixed name 'df.csv'.
    Overwrites existing files without asking (because, why not?).

    Parameters
    ----------
    data : pd.DataFrame
        The data to export.
    output_dir : str
        Directory where the file will be saved.
    """
    try:
        os.makedirs(output_dir, exist_ok=True)
        filepath = os.path.join(output_dir, "df.csv")

        # flatten MultiIndex columns (if any)
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.droplevel(0)

        data.to_csv(filepath, index=True)
        print(f"💾 Data successfully saved at: {filepath}")
        return filepath

    except Exception as e:
        print(f"❌ Failed to export data: {e}")
        return None


# usage example:
export_data(df_hist)



💾 Data successfully saved at: ../data/processed\df.csv


'../data/processed\\df.csv'